<a href="https://colab.research.google.com/github/mcabrera2004/DeepSeekTranslatorAI/blob/main/TranslatorAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
!chmod +x cuda_11.8.0_520.61.05_linux.run
!./cuda_11.8.0_520.61.05_linux.run --silent --toolkit

--2025-03-08 05:54:34--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda_11.8.0_520.61.05_linux.run
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 23.66.101.47, 23.66.101.38
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|23.66.101.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4336730777 (4.0G) [application/octet-stream]
Saving to: ‘cuda_11.8.0_520.61.05_linux.run’

cuda_11.8.0_520.61. 100%[===================>]   4.04G   236MB/s    in 36s     

2025-03-08 05:55:10 (115 MB/s) - ‘cuda_11.8.0_520.61.05_linux.run’ saved [4336730777/4336730777]



In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install pillow pdf2image python-docx

In [ ]:
!pip install torch==2.0.1+cu118 torchaudio==2.0.2+cu118 torchvision==0.15.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
!pip install git+https://github.com/deepseek-ai/DeepSeek-VL2.git

  Cloning https://github.com/deepseek-ai/DeepSeek-VL2.git to /tmp/pip-req-build-e9agflpj
  Running command git clone --filter=blob:none --quiet https://github.com/deepseek-ai/DeepSeek-VL2.git /tmp/pip-req-build-e9agflpj
  Resolved https://github.com/deepseek-ai/DeepSeek-VL2.git to commit ef9f91e2b6426536b83294c11742c27be66361b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from google.colab import files

# Subir el archivo PDF
uploaded = files.upload()

# Obtener el nombre del archivo subido
ruta_pdf = list(uploaded.keys())[0]
print(f"Archivo subido: {ruta_pdf}")

Saving Weir_Egg_removed.pdf to Weir_Egg_removed (1).pdf
Archivo subido: Weir_Egg_removed (1).pdf


In [1]:
import torch

def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print("Usando GPU para inferencia:", torch.cuda.get_device_name(0))
    else:
        device = torch.device("cpu")
        print("GPU no disponible, usando CPU para inferencia")
    return device

device = get_device()

Usando GPU para inferencia: Tesla T4


In [ ]:
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM

model_path = "deepseek-ai/deepseek-vl2-tiny"
vl_chat_processor = DeepseekVLV2Processor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# Cargar modelo - para GPU usamos half precision
print(f"Cargando modelo en {device}...")
#torch_dtype = torch.bfloat16 if device.type == "cuda" else torch.float32
torch_dtype = torch.float32
vl_gpt = DeepseekVLV2ForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch_dtype
)

# Mover el modelo al dispositivo
vl_gpt = vl_gpt.to(device)
vl_gpt.eval()  # Modo de inferencia
print("Modelo cargado")

Python version is above 3.10, patching the collections module.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Cargando modelo en cuda...


In [ ]:
from pdf2image import convert_from_path
import os

def pdf_a_imagenes(ruta_pdf, ruta_salida="imagenes_paginas"):
    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)

    paginas = convert_from_path(ruta_pdf, dpi=300)
    rutas_imagenes = []

    for i, pagina in enumerate(paginas):
        ruta_imagen = os.path.join(ruta_salida, f"pagina_{i+1}.jpg")
        pagina.save(ruta_imagen, "JPEG")
        rutas_imagenes.append(ruta_imagen)

    return rutas_imagenes

In [ ]:
from PIL import Image

def extraer_texto_con_deepseek(ruta_imagen):
    image = Image.open(ruta_imagen).convert("RGB")

    conversation = [
        {
            "role": "<|User|>",
            "content": "<image>\n|ref|>Extrae el texto de la imagen.<|/ref|>",
            "images": [image],
        },
        {"role": "<|Assistant|>", "content": ""},
    ]

    model_inputs = vl_chat_processor(
        conversations=conversation,
        images=[image],
        force_batchify=True,
        system_prompt=""
    ).to(device)  # Mover inputs al dispositivo configurado

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**model_inputs)

    with torch.no_grad():
        outputs = vl_gpt.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=model_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=2048,
            do_sample=False,
            use_cache=True
        )

    response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)
    return response

In [ ]:
def traducir_texto(texto):
    conversation = [
        {"role": "<|User|>", "content": f"Traduce el siguiente texto al español: {texto}"},
        {"role": "<|Assistant|>", "content": ""},
    ]

    model_inputs = vl_chat_processor(
        conversations=conversation,
        force_batchify=True,
        system_prompt=""
    ).to(device)  # Mover inputs al dispositivo configurado

    inputs_embeds = vl_gpt.prepare_inputs_embeds(**model_inputs)

    with torch.no_grad():
        outputs = vl_gpt.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=model_inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=2048,
            do_sample=False,
            use_cache=True
        )

    response = tokenizer.decode(outputs[0].tolist(), skip_special_tokens=True)
    return response

In [ ]:
from docx import Document

def traducir_y_guardar_en_word(ruta_pdf, nombre_archivo="libro_traducido.docx"):
    imagenes = pdf_a_imagenes(ruta_pdf)
    doc = Document()
    doc.add_heading("Libro Traducido", level=1)

    buffer = ""

    for i, imagen in enumerate(imagenes):
        print(f"🔍 Procesando página {i+1}...")
        texto_extraido = extraer_texto_con_deepseek(imagen)

        if buffer:
            texto_extraido = buffer + " " + texto_extraido

        if not texto_extraido.strip().endswith("."):
            partes = texto_extraido.rsplit(". ", 1)
            if len(partes) > 1:
                texto_extraido, buffer = partes[0] + ".", partes[1]
            else:
                buffer = texto_extraido
                texto_extraido = ""
        else:
            buffer = ""

        if texto_extraido.strip():
            traduccion = traducir_texto(texto_extraido)
            doc.add_paragraph(traduccion)

        doc.save(nombre_archivo)

    if buffer.strip():
        traduccion = traducir_texto(buffer)
        doc.add_paragraph(traduccion)
        doc.save(nombre_archivo)

    print(f"✅ Traducción completada. Guardado en {nombre_archivo}")

In [ ]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
if __name__ == "__main__":
    traducir_y_guardar_en_word(ruta_pdf)

🔍 Procesando página 1...


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


NotImplementedError: No operator found for `memory_efficient_attention_forward` with inputs:
     query       : shape=(10, 729, 16, 72) (torch.bfloat16)
     key         : shape=(10, 729, 16, 72) (torch.bfloat16)
     value       : shape=(10, 729, 16, 72) (torch.bfloat16)
     attn_bias   : <class 'NoneType'>
     p           : 0.0
`flshattF@v2.3.0` is not supported because:
    requires device with capability > (8, 0) but your GPU has capability (7, 5) (too old)
    bf16 is only supported on A100+ GPUs
`tritonflashattF` is not supported because:
    requires device with capability > (8, 0) but your GPU has capability (7, 5) (too old)
    bf16 is only supported on A100+ GPUs
    operator wasn't built - see `python -m xformers.info` for more info
    triton is not available
    requires GPU with sm80 minimum compute capacity, e.g., A100/H100/L4
`cutlassF` is not supported because:
    bf16 is only supported on A100+ GPUs
`smallkF` is not supported because:
    max(query.shape[-1] != val

In [ ]:
from google.colab import files

# Descargar el archivo DOCX generado
files.download("libro_traducido.docx")

FileNotFoundError: Cannot find file: libro_traducido.docx